In [1]:
# QUESTION 1: WHAT ARE THE PROBABILITIES?
# Training data
hidden_states = ['S','S','S','S','R','R','R','S','S','S','S','R','R','S','S']
observations = ['A','H','H','H','A','A','H','A','H','H','H','A','H','H','H']
prior_prob = {'S': 0, 'R': 0}
trans_prob = {'S->S': 0, 'S->R': 0, 'R->R': 0, 'R->S': 0}
emis_prob = {'S->H': 0, 'S->A': 0, 'R->H': 0, 'R->A': 0}

# Find emission probability
sh, sa, rh, ra = 0, 0, 0, 0
for i in range(len(hidden_states)-1):
    if hidden_states[i] is 'S' and observations[i] is 'H':
        sh += 1
    if hidden_states[i] is 'S' and observations[i] is 'A':
        sa += 1
    if hidden_states[i] is 'R' and observations[i] is 'H':
        rh += 1
    if hidden_states[i] is 'R' and observations[i] is 'A':
        ra += 1
emis_prob['S->H'] = sh/(sh + sa)
emis_prob['S->A'] = 1 - emis_prob['S->H']
emis_prob['R->H'] = rh/(rh + ra)
emis_prob['R->A'] = 1 - emis_prob['R->H']
emis_prob

{'S->H': 0.7777777777777778,
 'S->A': 0.2222222222222222,
 'R->H': 0.4,
 'R->A': 0.6}

In [2]:
# Find transmission probability
ss, sr, rr, rs = 0, 0, 0, 0
for i in range(len(hidden_states)-1):
    if hidden_states[i] is 'S' and hidden_states[i+1] is 'S':
        ss += 1
    if hidden_states[i] is 'S' and hidden_states[i+1] is 'R':
        sr += 1
    if hidden_states[i] is 'R' and hidden_states[i+1] is 'R':
        rr += 1
    if hidden_states[i] is 'R' and hidden_states[i+1] is 'S':
        rs += 1
trans_prob['S->S'] = ss/(ss + sr)
trans_prob['S->R'] = 1 - trans_prob['S->S']
trans_prob['R->R'] = rr/(rr + rs)
trans_prob['R->S'] = 1 - trans_prob['R->R']
trans_prob

{'S->S': 0.7777777777777778,
 'S->R': 0.2222222222222222,
 'R->R': 0.6,
 'R->S': 0.4}

In [3]:
# QUESTION 2: WHAT IS THE PROBABILITY OF A RANDOM DAY?
# Find prior probability
count_h = 0
count_a = 0
for s in hidden_states:
    if s is 'S':
        prior_prob['S'] += 1
    if s is 'R':
        prior_prob['R'] += 1
prior_prob['S'] /= len(hidden_states)
prior_prob['R'] = 1 - prior_prob['S']
prior_prob

{'S': 0.6666666666666666, 'R': 0.33333333333333337}

In [4]:
# QUESTION 3: IF ANGRY or HAPPY, WHAT IS THE PROBABILITY IT'S SUNNY or RAINY?
condition_a = {'S': sa/(sa+ra), 'R': ra/(sa+ra)}
condition_h = {'S': sh/(sh+rh), 'R': rh/(sh+rh)}
print('If Angry: ',condition_a, '\nIf Happy: ',condition_h)

If Angry:  {'S': 0.4, 'R': 0.6} 
If Happy:  {'S': 0.7777777777777778, 'R': 0.2222222222222222}


In [5]:
# QUESTION 4: IF FOR 3 DAYS [HAPPY, HAPPY, ANGRY], WHAT WAS THE WEATHER?
from IPython.core.debugger import set_trace
new_observations = ['H','H','A','A','A','H']
predict_states = []
weather = {}
weather['S'] = prior_prob['S']
weather['R'] = prior_prob['R']

# Viterbi algorithm
for i in range(len(new_observations)):
#     set_trace()
    w1 = weather['S']
    w2 = weather['R']
    if new_observations[i] is 'H':
        if i == 0:
            weather['S'] *= emis_prob['S->H']
            weather['R'] *= emis_prob['R->H']
        else:
            weather['S'] = max([w1 * emis_prob['S->H'] * trans_prob['S->S'],w2 * emis_prob['S->H'] * trans_prob['R->S']])
            weather['R'] = max([w1 * emis_prob['R->H'] * trans_prob['S->R'],w2 * emis_prob['R->H'] * trans_prob['R->R']])
        predict_states.append(max(weather, key=weather.get))
        continue
    if new_observations[i] is 'A':
        if i == 0:
            weather['S'] *= emis_prob['S->A']
            weather['R'] *= emis_prob['R->A']
        else:
            weather['S'] = max([w1 * emis_prob['S->A'] * trans_prob['S->S'],w2 * emis_prob['S->A'] * trans_prob['R->S']])
            weather['R'] = max([w1 * emis_prob['R->A'] * trans_prob['S->R'],w2 * emis_prob['R->A'] * trans_prob['R->R']])
        predict_states.append(max(weather, key=weather.get))

predict_states

['S', 'S', 'S', 'R', 'R', 'S']